In [ ]:
!pip install sentence-transformers==2.2.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 95.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=32edeb6d6e3836469f7991675399156bf16378879a4501a2c0a11f5d929257dd
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [ ]:
import os
import sys
sys.path.insert(1, "drive/MyDrive/colab_notebook_data/")
import data_bert
import sentence_transformers
import sentence_transformers.losses
import torch
import numpy as np
import pandas as pd
import config
from google.colab import runtime

In [ ]:
training_sentence_pairs = []

mean_topic_length = 0
mean_content_length = 0

total_topic_lengths = []
total_content_lengths = []

topics_text_info = pd.read_csv(config.resources_path + "topics_text_tree_information.csv", index_col=0)

for k in range(len(data_bert.has_correlation_train_contents)):
  content_num_id = data_bert.has_correlation_train_contents[k]
  topic_num_id = data_bert.has_correlation_test_topics[k]
  
  """if (type(data_bert.topics.iloc[topic_num_id]["title_translate"])==float and type(data_bert.topics.iloc[topic_num_id]["description_translate"])==float):
    # print("Problem: no textual info! ", topic_num_id)
    continue"""
  if type(topics_text_info.iloc[topic_num_id]["0"]) == float:
    continue
  if (type(data_bert.contents.iloc[content_num_id]["title_translate"])==float and type(data_bert.contents.iloc[content_num_id]["description_translate"])==float):
    # print("Problem: no textual info! ", content_num_id)
    continue
  
  if k % 10000 == 0:
      print("Loaded text info:", k, "   out of ", len(data_bert.has_correlation_train_contents))

  """if type(data_bert.topics.iloc[topic_num_id]["title_translate"])==float:
    topic_str = data_bert.topics.iloc[topic_num_id]["description_translate"]
  elif type(data_bert.topics.iloc[topic_num_id]["description_translate"])==float:
    topic_str = data_bert.topics.iloc[topic_num_id]["title_translate"]
  else:
    topic_str = data_bert.topics.iloc[topic_num_id]["title_translate"] + " " + data_bert.topics.iloc[topic_num_id]["description_translate"]"""
  topic_str = topics_text_info.iloc[topic_num_id]["0"]
  
  if type(data_bert.contents.iloc[content_num_id]["title_translate"])==float:
    content_str = data_bert.contents.iloc[content_num_id]["description_translate"]
  elif type(data_bert.contents.iloc[content_num_id]["description_translate"])==float:
    content_str = data_bert.contents.iloc[content_num_id]["title_translate"]
  else:
    content_str = data_bert.contents.iloc[content_num_id]["title_translate"] + " " + data_bert.contents.iloc[content_num_id]["description_translate"]
  
  mean_topic_length += len(topic_str.split())
  mean_content_length += len(content_str.split())

  total_topic_lengths.append(len(topic_str.split()))
  total_content_lengths.append(len(content_str.split()))

  if len(topic_str.split()) > 50:
    topic_str = " ".join(topic_str.split()[:50])
  if len(content_str.split()) > 50:
    content_str = " ".join(content_str.split()[:50])

  training_sentence_pairs.append(
    sentence_transformers.InputExample(texts=[
      topic_str, content_str
    ]))

print(mean_topic_length / len(data_bert.has_correlation_train_contents))
print(mean_content_length / len(data_bert.has_correlation_train_contents))

print(np.median(total_topic_lengths))
print(np.median(total_content_lengths))

Loaded text info: 0    out of  279919
Loaded text info: 10000    out of  279919
Loaded text info: 20000    out of  279919
Loaded text info: 30000    out of  279919
Loaded text info: 40000    out of  279919
Loaded text info: 50000    out of  279919
Loaded text info: 60000    out of  279919
Loaded text info: 70000    out of  279919
Loaded text info: 80000    out of  279919
Loaded text info: 90000    out of  279919
Loaded text info: 100000    out of  279919
Loaded text info: 110000    out of  279919
Loaded text info: 120000    out of  279919
Loaded text info: 130000    out of  279919
Loaded text info: 140000    out of  279919
Loaded text info: 150000    out of  279919
Loaded text info: 160000    out of  279919
Loaded text info: 170000    out of  279919
Loaded text info: 180000    out of  279919
Loaded text info: 190000    out of  279919
Loaded text info: 200000    out of  279919
Loaded text info: 210000    out of  279919
Loaded text info: 220000    out of  279919
Loaded text info: 230000 

In [ ]:
model = sentence_transformers.SentenceTransformer("drive/MyDrive/colab_notebook_data/st_models/mpnet_english")

In [ ]:
dataloader = torch.utils.data.DataLoader(training_sentence_pairs, shuffle=True, batch_size=50)
loss = sentence_transformers.losses.MultipleNegativesRankingLoss(model=model)
# num_epochs = 10
num_epochs = 10
warmup_steps = int(len(dataloader) * num_epochs * 0.1) #10% of train data

In [ ]:
if not os.path.isdir("drive/MyDrive/colab_notebook_data/st_models_out"):
  os.mkdir("drive/MyDrive/colab_notebook_data/st_models_out")
if not os.path.isdir("drive/MyDrive/colab_notebook_data/st_models_out/mpnet_english_tree"):
  os.mkdir("drive/MyDrive/colab_notebook_data/st_models_out/mpnet_english_tree")
if not os.path.isdir("drive/MyDrive/colab_notebook_data/st_models_out/mpnet_english_checkpoints_tree"):
  os.mkdir("drive/MyDrive/colab_notebook_data/st_models_out/mpnet_english_checkpoints_tree")
model.fit(train_objectives=[(dataloader, loss)],
          epochs=num_epochs,
          save_best_model = True,
          output_path='drive/MyDrive/colab_notebook_data/st_models_out/mpnet_english_tree',
          warmup_steps=warmup_steps,
          checkpoint_path="drive/MyDrive/colab_notebook_data/st_models_out/mpnet_english_checkpoints_tree",
          checkpoint_save_steps=1000)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5509 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5509 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5509 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5509 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/5509 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5509 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5509 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5509 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5509 [00:00<?, ?it/s]

In [ ]:
"""del model
del dataloader
torch.cuda.empty_cache()"""

'del model\ndel dataloader\ntorch.cuda.empty_cache()'

In [ ]:
print("Finished.")

Finished eeee


In [ ]:
runtime.unassign()